In [67]:
from tqdm.notebook import tqdm
import json
import pickle

In [68]:
# Collecting all the malayalam chars
malayalam_chars = [chr(code) for code in range(0x0D00, 0x0D7F + 1)]
malayalam_chars[:10], malayalam_chars[-10:], len(malayalam_chars)

# Creating a better version including full consonant × vowel sign syllable vocabulary (like ക, കാ, കി, കീ … for every consonant)
consonants = [chr(c) for c in range(0x0D15, 0x0D39 + 1)] # basic chars
vowel_signs = {
    "ാ": "aa",
    "ി": "i",
    "ീ": "ii",
    "ു": "u",
    "ൂ": "uu",
    "ൃ": "r̥",
    "െ": "e",
    "േ": "ee",
    "ൈ": "ai",
    "ൊ": "o",
    "ോ": "oo",
    "ൌ": "au",
}
syllables = []
for cons in consonants:
    for sign, label in vowel_signs.items():
        syllable = cons + sign
        syllables.append(syllable)

In [69]:
lengths = list(map(lambda x: len(list(x.encode())), syllables))
set(lengths)

{6}

In [70]:
# Creating a new mapping starting from 256 to encode all malayalam chars
malayalam_chars_mapping = {tuple(char.encode("utf-8")) : 256+i   for i, char in enumerate(malayalam_chars)}
print(f"length of malayalam chars : {len(malayalam_chars_mapping)}")
print(f"First entry: {sorted([(k,v) for k, v in malayalam_chars_mapping.items()], key=lambda x: x[1])[0]}")
print(f"Last entry: {sorted([(k,v) for k, v in malayalam_chars_mapping.items()], key=lambda x: x[1])[-1]}")
malayalam_syllabeles_mapping = {tuple(char.encode("utf-8")) : 256+len(malayalam_chars_mapping)+i   for i, char in enumerate(syllables)}
print(f"First entry: {sorted([(k,v) for k, v in malayalam_syllabeles_mapping.items()], key=lambda x: x[1])[0]}")
print(f"Last entry: {sorted([(k,v) for k, v in malayalam_syllabeles_mapping.items()], key=lambda x: x[1])[-1]}")

vocabulary = {v: bytes(k).decode() for k, v in malayalam_chars_mapping.items()}
vocabulary.update({v: bytes(k).decode() for k, v in malayalam_syllabeles_mapping.items()})

with open("../malayalam_chars_mapping.pkl", "wb") as f:
    pickle.dump(malayalam_chars_mapping, f)

with open("../malayalam_syllabeles_mapping.pkl", "wb") as f:
    pickle.dump(malayalam_syllabeles_mapping, f)

def decode_malayalam_char_ids(ids: list[int]) -> str:
    """Take a list of malayalam char ids and return the corresponding malayalam string"""
    return "".join([vocabulary.get(id) for id in ids])

length of malayalam chars : 128
First entry: ((224, 180, 128), 256)
Last entry: ((224, 181, 191), 383)
First entry: ((224, 180, 149, 224, 180, 190), 384)
Last entry: ((224, 180, 185, 224, 181, 140), 827)


In [71]:
def get_stats(ids: list[int]) -> dict[tuple[int, int], int]:
    """Takes a list of ints and find the occurance of each pair"""
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def get_top_pair(stats: dict) -> tuple[int, int]:
    """Find the most occuring pair"""
    return max(stats, key=stats.get)

In [72]:
def merge_malayalam_char_tokens(tokens: list[int]) -> list[int]:
    """Merge UTF-8 byte sequences into new vocab ids for Malayalam characters"""
    merged_tokens = []
    i = 0
    while i < len(tokens):
        if i + 2 < len(tokens):  # check if 3 bytes available
            key = (tokens[i], tokens[i+1], tokens[i+2])
            value = malayalam_chars_mapping.get(key)
            if value is not None:
                merged_tokens.append(value)
                i += 3
                continue
        # fallback: keep single byte
        merged_tokens.append(tokens[i])
        i += 1
    return merged_tokens

def merge_malayalam_syllabele_tokens(tokens: list[int]) -> list[int]:
    """Merge UTF-8 byte sequences into new vocab ids for Malayalam characters"""
    merged_tokens = []
    i = 0
    while i < len(tokens):
        if i + 5 < len(tokens):  # check if 6 bytes available syllabele is always 6 bytes
            key = (tokens[i], tokens[i+1], tokens[i+2], tokens[i+3], tokens[i+4], tokens[i+5])
            value = malayalam_syllabeles_mapping.get(key)
            if value is not None:
                merged_tokens.append(value)
                i += 6
                continue
        # fallback: keep single byte
        merged_tokens.append(tokens[i])
        i += 1
    return merged_tokens

In [73]:
text = "ഖി"
tokens = list(text.encode("utf-8"))
print(tokens)
updated_tokens = merge_malayalam_syllabele_tokens(tokens)
print(updated_tokens)
updated_tokens = merge_malayalam_char_tokens(updated_tokens)
print(updated_tokens)
print(decode_malayalam_char_ids(updated_tokens))

[224, 180, 150, 224, 180, 191]
[397]
[397]
ഖി


In [74]:
def merge_pair(tokens, pair_to_merge, new_idx):
    """Merge common pairs to create new pairs"""
    merged_tokens = []
    i = 0
    while i < len(tokens):
        if i < len(tokens)-1 and (tokens[i], tokens[i+1]) == pair_to_merge:
            merged_tokens.append(new_idx)
            i += 2
        else:
            merged_tokens.append(tokens[i])
            i += 1
    return merged_tokens

In [75]:
with open("../dataset/output.json", "r") as f:
    ds = json.load(f)

In [76]:
len(ds)

6050956

In [77]:
text = "".join(ds[:10000])
print(len(text))
print(text[:1000])


763524
അപകടങ്ങള്‍ നേരിടാന്‍ എപ്പോഴും സജ്ജരായിരിക്കണം.ഞങ്ങള്‍ തമ്മില്‍ വ്യക്തിപരമായ അഭിപ്രായവ്യത്യാസമൊന്നുമില്ല.സഹിക്കാന്‍ കയുന്നതിന് പരിധിയുണ്ട്.ഇന്ത്യന്‍ എംബസി ഇടപെട്ട കാരണമാണ് മോചനം സാധ്യമായത്.ഒരു കാര്യം ഓര്‍മിപ്പിക്കട്ടെ...12,816 എന്ന നിലയിൽ ഗാപ് അപ്പോടെ നിഫ്റ്റി തുറന്നു, വളരെ അധികം നേരം ഏകീകരിച്ചു.കമ്മീഷന്റെ അന്വ .പിതൃസഹോദരനും പാര്‍ട്ടി സംസ്ഥാന അധ്യക്ഷനുമായ ശിവ്പാല്‍ യാദവ് ഉള്‍പ്പെടെ നാലു പേരെ മുഖ്യമന്ത്രി അഖിലേഷ് യാദവ് മന്ത്രിസഭയില്‍ നിന്നു പുറത്താക്കിയതിന് തൊട്ടുപിന്നാലെ അഖിലേഷിന്‍റെ അനുയായിയും പാര്‍ട്ടി ജനറല്‍ സെക്രട്ടറിയുമായ രാംഗോപാല്‍ യാദവിനെ ശിവ്പാല്‍ യാദവ് ആറു വര്‍ഷത്തേക്ക് പാര്‍ട്ടിയില്‍ നിന്നു പുറത്താക്കിയതോടെയാണ് സമാജ്‌വാദി പാര്‍ട്ടിയില്‍ പ്രതിസന്ധി രൂക്ഷമായത്.അയാള്‍ പറഞ്ഞു എന്റെ അച്ഛനാണ് അയാളുടെ വീട്ടിലെ കര്‍മ്മങ്ങളെല്ലാം ചെയ്തിരുന്നതെന്ന്ആശുപത്രിയിൽ വച്ചാണ് ഇയാൾ മരിച്ചിരിക്കുന്നത്.മതി താപനില 80-90 ഡിഗ്രി ആണ്.ദില്ലി/വാഷിംഗ്ടണ്‍: ചൈനയുമായി അതിര്‍ത്തിയില്‍ നടക്കുന്ന സംഘര്‍ഷങ്ങളില്‍ മോദി സന്തുഷ്ടനല്ലെന്ന് ഡൊണാള്‍ഡ് ട്രംപ്കരുതിക്കൂട്ടി ചെയ്യുന്നതുമല്ല."""കുഴൽ വിഴുങ്ങിക്കോളൂ.

In [78]:
tokens = list(text.encode("utf-8")) # Text to list of ints between 0-255
l1 = len(tokens)
print(f"Orginal number of tokens : {len(tokens)}")
tokens = merge_malayalam_syllabele_tokens(tokens) 
l2 = len(tokens)
print(f"Number of tokens after merging  syllable vocabulary (like ക, കാ, കി, കീ …): {len(tokens)}, {(l1-l2)/l1:.2f}")
tokens = merge_malayalam_char_tokens(tokens)
l3 = len(tokens)
print(f"Number of tokens after merging  all malayalm chars: {len(tokens)}, {(l1-l3)/l1:.2f}, {(l2-l3)/l2:.2f}")

Orginal number of tokens : 2106330
Number of tokens after merging  syllable vocabulary (like ക, കാ, കി, കീ …): 1344545, 0.36
Number of tokens after merging  all malayalm chars: 636675, 0.70, 0.53


In [79]:
merged_chars_mapping = {}

final_vocab_size = 5000
current_vocab_size = 255 + len(malayalam_chars_mapping) + len(malayalam_syllabeles_mapping)
num_merges = final_vocab_size - current_vocab_size
print(f"New num of tokens : {num_merges}")
token_ids = list(tokens)

for i in tqdm(range(num_merges)):
    stats = get_stats(token_ids)
    pair = max(stats, key=stats.get)
    idx = current_vocab_size + 1 + i
    token_ids = merge_pair(token_ids, pair, idx)
    merged_chars_mapping[pair] = idx

New num of tokens : 4173


  0%|          | 0/4173 [00:00<?, ?it/s]

In [80]:
import pickle

with open("../merged_chars_mapping.pkl", "wb") as f:
    pickle.dump(merged_chars_mapping, f)


In [81]:
reverse_merged_chars_mapping = {v: k for k, v in merged_chars_mapping.items()}

def decode_merged_chars_mapping(token_id: int) -> str:

    if token_id < 256:
        return chr(token_id)
    
    if token_id in vocabulary:
        return vocabulary[token_id]
    
    id1, id2 = reverse_merged_chars_mapping.get(token_id)
    char1 = decode_merged_chars_mapping(id1)
    char2 = decode_merged_chars_mapping(id2)
    return char1 + char2
    

decode_merged_chars_mapping(2000)

'ത്ഥ'

In [82]:
new_vocabulary = {v: decode_merged_chars_mapping(v) for v in merged_chars_mapping.values()}
vocabulary.update(new_vocabulary)

In [83]:
with open("../vocabulary.pkl", "wb") as f:
    pickle.dump(vocabulary, f)
